In [1]:
import torch
import numpy
import numpy as np
import pandas as pd
import json

from src.model import build_model
from src.utils import convert_tensor, evaluate_model

In [2]:
import numpy
tt = numpy.zeros((94129, 3194))
tt = tt.astype(np.float32)
tt = torch.from_numpy(tt)

In [3]:
with open('./data/hyperparameter.json') as fp:
    hparam = json.load(fp)

In [4]:
test_data = pd.read_pickle('./savepoints/0/test_data.pkl')
x = test_data[0]
y = test_data[1]

In [5]:
SS_mat = pd.read_pickle('./data/structural_similarity_matrix.pkl')
TS_mat = pd.read_pickle('./data/target_similarity_matrix.pkl')
GS_mat = pd.read_pickle('./data/GO_similarity_matrix.pkl')

In [6]:
mlb = pd.read_pickle('./data/mlb.pkl')
idx2label = pd.read_pickle('./data/idx2label.pkl')

In [7]:
SS, TS, GS, y = convert_tensor(x, y, SS_mat, TS_mat, GS_mat, mlb, idx2label)
y = y.int().numpy()

In [9]:
model = build_model(hparam)
model.load_model('./savepoints/0/model_checkpoint')
model.eval()

build_model(
  (encoder1): Sequential(
    (0): Linear(in_features=3194, out_features=1000, bias=True)
    (1): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.3, inplace=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=1000, out_features=200, bias=True)
    (5): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.3, inplace=True)
    (7): ReLU(inplace=True)
  )
  (decoder1): Sequential(
    (0): Linear(in_features=200, out_features=1000, bias=True)
    (1): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=1000, out_features=3194, bias=True)
    (4): Sigmoid()
  )
  (encoder2): Sequential(
    (0): Linear(in_features=3194, out_features=1000, bias=True)
    (1): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.3, inplace=True)
   

In [10]:
with torch.no_grad():
    _, _, _, pred = model(SS, TS, GS)
    pred = torch.sigmoid(pred) > 0.5    
    pred = pred.int().numpy()

In [11]:
acc, ma_rc, ma_pc, mi_rc, mi_pc = evaluate_model(y, pred)

In [12]:
print(f'Accuracy: {acc}\nMacro recall: {ma_rc}\nMacro_precision: {ma_pc}\nMicro_recall: {mi_rc}\nMicro_precision: {mi_pc}')

Accuracy: 0.9894187763601016
Macro recall: 0.9894594683940743
Macro_precision: 0.9815660016498825
Micro_recall: 0.9912265812310369
Micro_precision: 0.990218105513046
